In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import ast
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import joblib
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/itsdakshjain/Movie-Recommendation-System/refs/heads/main/data/movies_metadata.csv')

In [3]:
df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [4]:
df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [6]:
df.shape

(45466, 24)

In [7]:
df.isnull().sum()

,0
adult,0
belongs_to_collection,40972
budget,0
genres,0
homepage,37684
id,0
imdb_id,17
original_language,11
original_title,0
overview,954


In [8]:
df.duplicated().sum() # there are 13 duplicates in overall dataset

np.int64(13)

In [9]:
df = df.drop_duplicates().reset_index(drop = True) # drop them and making a new index as previous ones can have gaps

In [10]:
df = df[['title', 'overview', 'genres','tagline','vote_average','popularity']]#using selected columns

In [11]:
df

,title,overview,genres,tagline,vote_average,popularity
0,Toy Story,"Led by Woody, Andy's toys live happily in his ...","[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",NaN,7.7,21.946943
1,Jumanji,When siblings Judy and Peter discover an encha...,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",Roll the dice and unleash the excitement!,6.9,17.015539
2,Grumpier Old Men,A family wedding reignites the ancient feud be...,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",Still Yelling. Still Fighting. Still Ready for...,6.5,11.7129
3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...","[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",Friends are the people who let you be yourself...,6.1,3.859495
4,Father of the Bride Part II,Just when George Banks has recovered from his ...,"[{'id': 35, 'name': 'Comedy'}]",Just When His World Is Back To Normal... He's ...,5.7,8.387519
...,...,...,...,...,...,...
45448,Subdue,Rising and falling between a man and woman.,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",Rising and falling between a man and woman,4.0,0.072051
45449,Century of Birthing,An artist struggles to finish his work while a...,"[{'id': 18, 'name': 'Drama'}]",NaN,9.0,0.178241
45450,Betrayal,"When one of her hits goes wrong, a professiona...","[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",A deadly game of wits.,3.8,0.903007
45451,Satan Triumphant,"In a small town live two brothers, one a minis...",[],NaN,0.0,0.003503


In [12]:
df.isnull().sum()

,0
title,6
overview,954
genres,0
tagline,25045
vote_average,6
popularity,5


In [13]:
df = df.dropna(subset=['title']) # dropping titles as no use we can't do anything without name

In [14]:
df['overview'] = df['overview'].fillna('') # filling overview with blank

In [15]:
df.iloc[0]['genres'] #genres is in form of dictionary inside list , which as whole is string so use ast(abtract string tree)

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [16]:
df['genres'] = df['genres'].apply(lambda x :" ".join([i['name'] for i in ast.literal_eval(x)])) #ast.literal_eval(x):Takes the messy string and turns it into a real Python list of dictionaries.

In [17]:
df.head()

,title,overview,genres,tagline,vote_average,popularity
0,Toy Story,"Led by Woody, Andy's toys live happily in his ...",Animation Comedy Family,NaN,7.7,21.946943
1,Jumanji,When siblings Judy and Peter discover an encha...,Adventure Fantasy Family,Roll the dice and unleash the excitement!,6.9,17.015539
2,Grumpier Old Men,A family wedding reignites the ancient feud be...,Romance Comedy,Still Yelling. Still Fighting. Still Ready for...,6.5,11.7129
3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",Comedy Drama Romance,Friends are the people who let you be yourself...,6.1,3.859495
4,Father of the Bride Part II,Just when George Banks has recovered from his ...,Comedy,Just When His World Is Back To Normal... He's ...,5.7,8.387519


In [18]:
df['tagline'] = df['tagline'].fillna('') # filling tagline with blank

In [19]:
df.isnull().sum() # by dropping title's null, vote nd popularity's null also got removed

,0
title,0
overview,0
genres,0
tagline,0
vote_average,0
popularity,0


In [20]:
df['tags'] = df['overview']+ " " + df['genres'] +" "+ df['tagline']

In [21]:
df.head()

,title,overview,genres,tagline,vote_average,popularity,tags
0,Toy Story,"Led by Woody, Andy's toys live happily in his ...",Animation Comedy Family,,7.7,21.946943,"Led by Woody, Andy's toys live happily in his ..."
1,Jumanji,When siblings Judy and Peter discover an encha...,Adventure Fantasy Family,Roll the dice and unleash the excitement!,6.9,17.015539,When siblings Judy and Peter discover an encha...
2,Grumpier Old Men,A family wedding reignites the ancient feud be...,Romance Comedy,Still Yelling. Still Fighting. Still Ready for...,6.5,11.7129,A family wedding reignites the ancient feud be...
3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",Comedy Drama Romance,Friends are the people who let you be yourself...,6.1,3.859495,"Cheated on, mistreated and stepped on, the wom..."
4,Father of the Bride Part II,Just when George Banks has recovered from his ...,Comedy,Just When His World Is Back To Normal... He's ...,5.7,8.387519,Just when George Banks has recovered from his ...


In [22]:
df['tags'][1] # model can't work on text so use nlp

"When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- into their living room. Alan's only hope for freedom is to finish the game, which proves risky as all three find themselves running from giant rhinoceroses, evil monkeys and other terrifying creatures. Adventure Fantasy Family Roll the dice and unleash the excitement!"

In [23]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [24]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [25]:
def preprocess_text(text):
  text = str(text).lower() #lowercase
  text = re.sub(r'[^a-zA-Z\s]', '', text) #removed the punctuations

  words = text.split() # tokenizing (.split() does the exact same job as word_tokenize but much faster, word_tokenize looks for punctuation and divide but we had already remove punctuation in previous step)
  words = [word for word in words if word not in stop_words] #remove stopwords

  words = [lemmatizer.lemmatize(word) for word in words] #lemmatize

  return " ".join(words)



In [26]:
df['tags'] = df['tags'].apply(preprocess_text)

In [27]:
df

,title,overview,genres,tagline,vote_average,popularity,tags
0,Toy Story,"Led by Woody, Andy's toys live happily in his ...",Animation Comedy Family,,7.7,21.946943,led woody andys toy live happily room andys bi...
1,Jumanji,When siblings Judy and Peter discover an encha...,Adventure Fantasy Family,Roll the dice and unleash the excitement!,6.9,17.015539,sibling judy peter discover enchanted board ga...
2,Grumpier Old Men,A family wedding reignites the ancient feud be...,Romance Comedy,Still Yelling. Still Fighting. Still Ready for...,6.5,11.7129,family wedding reignites ancient feud nextdoor...
3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",Comedy Drama Romance,Friends are the people who let you be yourself...,6.1,3.859495,cheated mistreated stepped woman holding breat...
4,Father of the Bride Part II,Just when George Banks has recovered from his ...,Comedy,Just When His World Is Back To Normal... He's ...,5.7,8.387519,george bank recovered daughter wedding receive...
...,...,...,...,...,...,...,...
45448,Subdue,Rising and falling between a man and woman.,Drama Family,Rising and falling between a man and woman,4.0,0.072051,rising falling man woman drama family rising f...
45449,Century of Birthing,An artist struggles to finish his work while a...,Drama,,9.0,0.178241,artist struggle finish work storyline cult pla...
45450,Betrayal,"When one of her hits goes wrong, a professiona...",Action Drama Thriller,A deadly game of wits.,3.8,0.903007,one hit go wrong professional assassin end sui...
45451,Satan Triumphant,"In a small town live two brothers, one a minis...",,,0.0,0.003503,small town live two brother one minister one h...


In [28]:
df = df.reset_index(drop = True) # again making a new index as previous ones can have gaps

In [29]:
indices = pd.Series(df.index,index = df['title']).drop_duplicates()
indices

,0
title,
Toy Story,0
Jumanji,1
Grumpier Old Men,2
Waiting to Exhale,3
Father of the Bride Part II,4
...,...
Subdue,45442
Century of Birthing,45443
Betrayal,45444


##Vectorization

### TfidfVectorizer

* TfidfVectorizer is better than One Hot Encodding (OHE) and BagOfWords (BOG)

###

In [30]:
tfidf = TfidfVectorizer(max_features=50000,ngram_range=(1,2),stop_words='english') #we have already used stop words but still writting again no issue

In [31]:
tfidf_matrix = tfidf.fit_transform(df['tags'])

In [32]:
tfidf_matrix

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1548114 stored elements and shape (45447, 50000)>

### Cosine Similarity


In [33]:
def recommendation(title, n = 10):
  if title not in indices:
    return ['Movie not found']

  idx = indices[title]
  similarity_score = cosine_similarity(tfidf_matrix[idx], tfidf_matrix).flatten()
  similar_idx = similarity_score.argsort()[::-1][1:n+1]
  return df['title'].iloc[similar_idx]



In [34]:
recommendation('Toy Story',5)

,title
2996,Toy Story 2
15344,Toy Story 3
24512,Small Fry
28972,Superstar Goofy
17184,Group Sex


## Pickles for streamlit

In [35]:
joblib.dump(tfidf_matrix, 'tfidf_matrix.pkl')
joblib.dump(indices, 'indices.pkl')
joblib.dump(tfidf, 'tfidf.pkl')

df.to_pickle('df.pkl')
